In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from optiver_features_handler import get_features_map_for_stock, get_row_id

In [2]:
DATA_DIRECTORY = os.path.join("..","input","optiver-realized-volatility-prediction")
TRADE_TRAIN_DIRECTORY = os.path.join(DATA_DIRECTORY,"trade_train.parquet")
TRADE_TEST_DIRECTORY = os.path.join(DATA_DIRECTORY,"trade_test.parquet")
BOOK_TRAIN_DIRECTORY = os.path.join(DATA_DIRECTORY,"book_train.parquet")
BOOK_TEST_DIRECTORY = os.path.join(DATA_DIRECTORY,"book_test.parquet")
OUTPUT_DIRECTORY = os.path.join("..","output")
os.makedirs(OUTPUT_DIRECTORY,exist_ok=True)

In [3]:
train_df = pd.read_csv(os.path.join(DATA_DIRECTORY,"train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIRECTORY,"test.csv"))

In [4]:
data_interval_seconds = 5
data_intervals_count = int(600/data_interval_seconds)
class OptiverRealizedVolatilityDataset(Dataset):
    def __init__(self, data_directory, mode="train", lazy_load=True):
        """initializes Optiver Competition dataset
        `mode`: train|test
        `data_directory`: the datadirectory of the input data, where there are test.csv, train.csv, and parquet folders for trade_train.parquet and other relevant folders
        """
        print("INIT: OptiverRealizedVolatilityDataset")
        if mode.lower() not in ['train','test']:
            raise Exception("Invalid mode passed for Optiver dataset. Valid values:train|test")
        self.data_directory = data_directory
        self.mode = mode.lower()
        self.main_df = pd.read_csv(os.path.join(self.data_directory,f'{self.mode}.csv'))
#         if self.mode == 'train':
#             self.main_df['row_id'] = self.main_df.apply(lambda x: f"{x['stock_id']:.0f}-{x['time_id']:.0f}", axis=1)
        if self.mode == 'test':
            self.main_df['target'] = 0
        
        self.cache_stocks_done_set = set()
        # this is our final features lookup where we park all our features which can be addressed by row_id
        # which is individual train/test.csv row id using 'stock_id`-`time_id`
        self.cache_rowid_feature_map = {}
        row_id_series = self.main_df['stock_id'].astype(str) + "-" +self.main_df['time_id'].astype(str)
        targets = self.main_df['target'].tolist()
        self.stock_possible_timeids_list = {}
        for idx, row_id in enumerate(row_id_series.tolist()):
            stock_id = int(row_id.split('-')[0])
            time_id = int(row_id.split('-')[1])
            self.cache_rowid_feature_map[row_id] = {'target':targets[idx], 'stock_id':stock_id,'time_id':time_id,'row_id':row_id}
            
            # below code is to make sure what timeids we expect from stock data extractor
            # in case of missing parquet files we'll have to know the keys to fill default values into
            if stock_id not in self.stock_possible_timeids_list:
                self.stock_possible_timeids_list[stock_id] = []
            self.stock_possible_timeids_list[stock_id].append(time_id)
            
        
        if lazy_load == False:
            worker_data = []
            for gkey, gdf in self.main_df.groupby(['stock_id']):
                worker_data.append((self.data_directory, self.mode, gkey))
#             print("---------- CPU COUNG:", multiprocessing.cpu_count())
            # NOTE: this was hell of a hunt; this windows and pytorch and jupyter combination is too tedious
            #       make sure the function that we distribute don't call pytorch
            chunksize = multiprocessing.cpu_count() * 1
            processed = 0
            for worker_data_chunk in [worker_data[i * chunksize:(i + 1) * chunksize] for i in range((len(worker_data) + chunksize - 1) // chunksize )]:
                with Pool(multiprocessing.cpu_count()) as p:
                    
                    feature_set_list = p.starmap(get_features_map_for_stock, worker_data_chunk)
                    
                    for feature_map in feature_set_list:
                        for rowid, features_dict in feature_map.items():
                            for fkey,fval in features_dict.items():
                                self.cache_rowid_feature_map[rowid][fkey] = fval
                            self.cache_rowid_feature_map[rowid]  = OptiverRealizedVolatilityDataset.transform_to_01_realized_volatility_linear_data(self.cache_rowid_feature_map[rowid])
                        # udpate the indications that we've already fetched this stock and the lazy loader code won't fetch this again
                        self.cache_stocks_done_set.add(int(rowid.split('-')[0]))
                    
                    processed += chunksize
                    print(f"Processed and loaded {processed} stocks features.")
    
    def __cache_generate_features(self, main_stock_id, main_time_id):
            
            main_row_id = get_row_id(main_stock_id, main_time_id)
            if main_stock_id not in self.cache_stocks_done_set:
#                 trade_df = pd.read_parquet(os.path.join(self.data_directory, f"trade_{self.mode}.parquet", f"stock_id={stock_id}"))   
                # we'll combine the featureset with the bigger feature set of all stocks
                feature_map = get_features_map_for_stock(self.data_directory, self.mode, main_stock_id)
                # NOTE: sometime we might now have parquet files in that case we'll have 3 entried in .csv while only 1 gets returned in feature map
                # we need to cover for that disparity
                for time_id in self.stock_possible_timeids_list[main_stock_id]:
                    expected_row_id = get_row_id(main_stock_id, time_id)
                    if expected_row_id not in feature_map:
                        feature_map[expected_row_id] = {}
                for rowid, features_dict in feature_map.items():
                    for fkey,fval in features_dict.items():
                        self.cache_rowid_feature_map[rowid][fkey] = fval
                    self.cache_rowid_feature_map[rowid]  = OptiverRealizedVolatilityDataset.transform_to_01_realized_volatility_linear_data(self.cache_rowid_feature_map[rowid])
                self.cache_stocks_done_set.add(main_stock_id)
#             print(self.cache_rowid_feature_map[main_row_id])
#             print(torch.tensor([self.cache_rowid_feature_map[main_row_id].get('book_realized_volatility',0)]))
#             print(torch.tensor(self.cache_rowid_feature_map[main_row_id].get('log_return1_2s', [0]*(int(600/2)))))
#             print(torch.tensor(self.cache_rowid_feature_map.get('book_directional_volume1_2s', [0]*(int(600/2)))))
            return self.cache_rowid_feature_map[main_row_id]
        
    @staticmethod
    def transform_to_01_realized_volatility_linear_data(features_dict):
        return (
                {
                    'row_id':features_dict['row_id'],
                    'stock_id':torch.tensor(features_dict['stock_id'], dtype=torch.float32),
#                     'book_realized_volatility':torch.tensor([features_dict.get('book_realized_volatility',0)]),
                    # TRADE FEATURES
                    'logrett_xs': torch.tensor(np.nan_to_num(features_dict.get('logrett_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'trade_volume_xs': torch.tensor(np.nan_to_num(features_dict.get('trade_volume_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'trade_ordercount_xs': torch.tensor(np.nan_to_num(features_dict.get('trade_ordercount_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    
                    'trade_price_min': torch.tensor(np.nan_to_num(features_dict.get('trade_price_min', 0)), dtype=torch.float32),
                    'trade_price_mean': torch.tensor(np.nan_to_num(features_dict.get('trade_price_mean', 0)), dtype=torch.float32),
                    'trade_price_max': torch.tensor(np.nan_to_num(features_dict.get('trade_price_max', 0)), dtype=torch.float32),
                    'trade_money_turnover_mean': torch.tensor(np.nan_to_num(features_dict.get('trade_money_turnover_mean', 0)), dtype=torch.float32),
                    'trade_money_turnover_std': torch.tensor(np.nan_to_num(features_dict.get('trade_money_turnover_std', 0)), dtype=torch.float32),
                    # BOOK FEATURES
                    'logret1_xs': torch.tensor(np.nan_to_num(features_dict.get('logret1_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'logret2_xs': torch.tensor(np.nan_to_num(features_dict.get('logret2_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_directional_volume1_xs': torch.tensor(np.nan_to_num(features_dict.get('book_directional_volume1_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_directional_volume2_xs': torch.tensor(np.nan_to_num(features_dict.get('book_directional_volume2_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_price_spread1_xs': torch.tensor(np.nan_to_num(features_dict.get('book_price_spread1_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_price_spread2_xs': torch.tensor(np.nan_to_num(features_dict.get('book_price_spread2_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_bid_spread_xs': torch.tensor(np.nan_to_num(features_dict.get('book_bid_spread_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_ask_spread_xs': torch.tensor(np.nan_to_num(features_dict.get('book_ask_spread_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_total_volume_xs': torch.tensor(np.nan_to_num(features_dict.get('book_total_volume_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    'book_volume_imbalance_xs': torch.tensor(np.nan_to_num(features_dict.get('book_volume_imbalance_xs', [0]*(int(600/data_interval_seconds)))), dtype=torch.float32),
                    
                    'book_price_min': torch.tensor(np.nan_to_num(features_dict.get('book_price_min', 0)), dtype=torch.float32),
                    'book_price_mean': torch.tensor(np.nan_to_num(features_dict.get('book_price_mean', 0)), dtype=torch.float32),
                    'book_price_max': torch.tensor(np.nan_to_num(features_dict.get('book_price_max', 0)), dtype=torch.float32),
                    'book_money_turnover_mean': torch.tensor(np.nan_to_num(features_dict.get('book_money_turnover_mean', 0)), dtype=torch.float32),
                    'book_money_turnover_std': torch.tensor(np.nan_to_num(features_dict.get('book_money_turnover_std', 0)), dtype=torch.float32),
                    
#                     'askp2_1s':torch.tensor(features_dict.get('askp2_1s', [0]*(int(600/1)))),
#                     'book_directional_volume1_1s':torch.tensor(features_dict.get('book_directional_volume1_1s', [0]*(int(600/1)))) 
                },
                torch.tensor([features_dict['target']])
#                 [features_dict['target']]
        )
    
    def __len__(self):
        return len(self.main_df)
    
    def __getitem__(self, idx):
        #TODO: handle for num_workers more than 0
        #      using https://pytorch.org/docs/stable/data.html
        #      using torch.util.data.get_worker_info()
        if torch.is_tensor(idx):
            idx = idx.tolist()
        stock_id = self.main_df.at[idx, 'stock_id']
        time_id = self.main_df.at[idx, 'time_id']
        x,y = self.__cache_generate_features(stock_id,time_id)
#         x, y = self.__transform_to_01_realized_volatility_linear_data(features_dict)
        return x,y

In [5]:
# dataset = OptiverRealizedVolatilityDataset(DATA_DIRECTORY, mode="train")
dataset = OptiverRealizedVolatilityDataset(DATA_DIRECTORY, mode="test", lazy_load=True)

INIT: OptiverRealizedVolatilityDataset


In [6]:
dataset[0]

({'row_id': '0-4',
  'stock_id': tensor(0.),
  'logrett_xs': tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00, -1.7399e-05, -1.7280e-05,
          -1.7399e-05, -1.7280e-05, -1.7400e-05, -1.7400e-05, -1.7281e-05,
          -1.7401e-05, -1.7282e-05, -1.7401e-05, -1.7283e-05, -1.7402e-05,
          -1.7402e-05, -1.7283e-05, -1.7403e-05, -1.7284e-05, -1.7404e-05,
           3.3377e-06,  3.2185e-06,  3.3377e-06,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.00

In [7]:
use_cuda = torch.cuda.is_available()
# use_cuda = False
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
model = None


def loss_fn_mse(y, pred):
    return torch.mean(torch.square((y-pred)))

def loss_fn_mspe(y, pred):
    return torch.mean(torch.square((y-pred)/y))

def loss_fn_orig(y, pred):
    return torch.sqrt(torch.mean(torch.square((y-pred)/y)))

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, mode='both', use_stock_id = True):
        super(NeuralNetwork, self).__init__()
        self.use_stock_id = use_stock_id
        self.flatten = nn.Flatten()
        self.mode = mode
        self.cnn_stack = nn.Sequential(
            nn.Conv1d(7, 24, kernel_size=6, stride=2, padding=0),
            nn.GELU(),
            nn.Conv1d(24, 32, kernel_size=4, stride=2, padding=0),
            nn.GELU(),
            nn.Conv1d(32, 48, kernel_size=3, stride=1, padding=0),
            nn.GELU(),
            nn.Conv1d(48, 64, kernel_size=2, stride=1, padding=0),

        )
        self.linear_stack = nn.Sequential(
            nn.Linear(1600, 1024),
            nn.GELU(),
#             nn.Dropout(0.3),
            nn.Linear(1024, 1024),
            nn.GELU(),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Linear(512, 256),
            nn.GELU(),

        )
        self.linear_hybrid = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

        
    def forward(self, feature_dict):
#         logits = self.basic_stack(x)
#         x = self.flatten(x)
        x = torch.cat([
#                             feature_dict['logrett_xs'].to(device)*10000, 
#                                torch.log(feature_dict['trade_volume_xs'].to(device)+0.001),
#                               torch.log(feature_dict['trade_ordercount_xs'].to(device)+0.001),
#                             feature_dict['trade_volume_xs'].to(device),
#                                         feature_dict['trade_ordercount_xs'].to(device),
#                                         feature_dict['book_total_volume_xs'].to(device),
#                                         feature_dict['book_volume_imbalance_xs'].to(device)
#                             feature_dict['logrett_xs'].to(device)*10000, 
                               torch.log(feature_dict['trade_volume_xs'].to(device)+0.001),
                              torch.log(feature_dict['trade_ordercount_xs'].to(device)+0.001),
                                feature_dict['logret1_xs'].to(device)*10000,
                            
                                 feature_dict['logret2_xs'].to(device)*10000,
                                    feature_dict['book_price_spread1_xs'].to(device)*10000, 
#                                 feature_dict['book_price_spread2_xs'].to(device)*10000, 
#                                 feature_dict['book_bid_spread_xs'].to(device)*10000, 
#                                 feature_dict['book_ask_spread_xs'].to(device)*10000, 
#                                  feature_dict['book_directional_volume1_xs'].to(device),
#                                 feature_dict['book_price_spread1_xs'].to(device)*1000,
                                torch.log(feature_dict['book_total_volume_xs'].to(device)+0.001),
                                torch.log(feature_dict['book_volume_imbalance_xs'].to(device)+0.001),
# #                              feature_dict['book_dirvolume_xs'],
                          ], 1)

#         x = torch.nan_to_num(feature_dict['logrett_xs']).type(torch.cuda.FloatTensor)
        
        
#         print(x)
#         input()
#         if torch.isnan(x).any():
# #             print(x)
#             print(feature_dict)
#             input()
        x = x.type(torch.cuda.FloatTensor)
        x = x.to(device)
        x = x.reshape(-1,7,data_intervals_count)
        
        logits = self.cnn_stack(x)
        logits = self.flatten(logits)
        
       
        
        logits = self.linear_stack(logits)
#         logits = torch.cat( [logits, torch.log(feature_dict['trade_money_turnover_mean'].type(torch.cuda.FloatTensor).to(device).reshape(-1,1)+0.001), 
#                                        torch.log(feature_dict['trade_price_mean'].type(torch.cuda.FloatTensor).to(device).reshape(-1,1)+0.001),
#                                        torch.log(feature_dict['book_money_turnover_mean'].type(torch.cuda.FloatTensor).to(device).reshape(-1,1)+0.001),
#                                        torch.log(feature_dict['book_price_mean'].type(torch.cuda.FloatTensor).to(device).reshape(-1,1)+0.001)
#                                       ], 1)
        
        if self.use_stock_id:
            stock_id = torch.tensor(feature_dict['stock_id']).reshape(-1,1)
            stock_id = stock_id.to(device)
            logits = torch.cat([logits, stock_id], 1)
            
        logits = self.linear_hybrid(logits)
        return logits





In [9]:
model = NeuralNetwork()
modelpath = "../input/optiver-realized-volatility-binarysentient-pytorch/07_1s_logret1n2_cnn_epoch_400_tloss_0.2393.pth"
modelpath = "../output/models/08_ultimate_hybridEXPX5_1LGRU_5s_StkFalse_0.0001_2_epoch_1_tloss_0.4187.pth"
model.load_state_dict(torch.load(modelpath))
model.to(device)
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (cnn_stack): Sequential(
    (0): Conv1d(7, 24, kernel_size=(6,), stride=(2,))
    (1): GELU()
    (2): Conv1d(24, 32, kernel_size=(4,), stride=(2,))
    (3): GELU()
    (4): Conv1d(32, 48, kernel_size=(3,), stride=(1,))
    (5): GELU()
    (6): Conv1d(48, 64, kernel_size=(2,), stride=(1,))
  )
  (linear_stack): Sequential(
    (0): Linear(in_features=1600, out_features=1024, bias=True)
    (1): GELU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): GELU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): GELU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): GELU()
  )
  (linear_hybrid): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [10]:
dataloader = DataLoader(dataset, batch_size=3,
                        shuffle=True, num_workers=0, pin_memory=True)
size = len(dataloader.dataset)
submission_data = []
data_interval_len = int(600/1)
output_scaling = 10000
data_ohlc_sample_len = 1 # 1 for each of open high low close
for batch, (Feature_X, feature_y) in enumerate(dataloader):
    row_ids = Feature_X['row_id']
    y = feature_y.to(device) * output_scaling 
    
    pred = model(Feature_X) 
#     print(pred)
    predicted_volatility = (pred/1000).tolist()
    for idx, row_id in enumerate(row_ids):
        submission_data.append({'row_id':row_id, 'target':predicted_volatility[idx][0]})
submission_df = pd.DataFrame(submission_data)
submission_df = dataset.main_df.merge(submission_df,on='row_id',how='left')
submission_df = submission_df.rename(columns={'target_y':'target'})
# submission_df
# print(submission_df.columns)
submission_df[['row_id','target']].to_csv("submission.csv", index=False)
# for idx, (X,y) in enumerate(dataset):
#     print(idx, X)

C:\Users\nisha\miniconda3\envs\bsstonks\lib\site-packages\ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [11]:
pd.read_csv("submission.csv")

,row_id,target
0,0-4,0.020016
1,0-32,0.007832
2,0-34,0.007832


In [12]:
import torch.version

In [13]:
torch.__version__

'1.9.0+cu111'